In [5]:
from selenium import webdriver as wd
import time
import pandas as pd 
import random
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [2]:
chrome_options = wd.ChromeOptions()
chrome_options.add_argument("--incognito")

driver = wd.Chrome('./tool/chromedriver.exe', chrome_options=chrome_options)
driver.implicitly_wait(3)

In [ ]:
target_year_ls = ['2015', '2016', '2017', '2018', '2019', '2020']
target_month_ls = ['03', '04', '05', '06', '07', '08', '09', '10', '11']
target_day_ls = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']

In [ ]:
urls = ['http://www.kbreport.com/teams/main?rows=20&order=TPCT&orderType=DESC&teamId=&defense_no=&year_from=2015&year_to=2020&split01=day&split02_1={target_year}-{target_month}-{target_day}&split02_2={target_year}-{target_month}-{target_day}']
    
limit = 20

def get_sublist_href(url: str):
    namu_link = []
    request = requests.get(url)
    sleep(1)
    
    parsed_html = BeautifulSoup(request.text, 'html.parser')
    a_element_tags = parsed_html.find_all('div', attrs={'class' : 'test'})
    for tag in a_element_tags:
        for link in tag.find_all('a'):
            namu_link.append(url + link['href'])
    
    namu_link = namu_link[:limit]
    print('Number of site: ', len(namu_link))
    return namu_link

def do_html_crawl(url: str):
    request = requests.get(url)
    sleep(1)
    parsed_html = BeautifulSoup(request.text, 'html.parser')
    return parsed_html

def do_process_with_thread_crawl(url: str):
    do_thread_crawl(get_sublist_href(url))
    
def do_thread_crawl(urls: list):
    thread_list = []
    with ThreadPoolExecutor(max_workers=8) as executor:
        for url in urls:
            thread_list.append(executor.submit(do_html_crawl, url))
        for execution in concurrent.futures.as_completed(thread_list):
            execution.result()

In [ ]:
start_time = time.time()

with Pool(processes=4) as pool:  
    pool.map(do_process_with_thread_crawl, urls)
    print("--- elapsed time %s seconds ---" % (time.time() - start_time))

In [6]:
driver.close()
driver.quit()